# A Priori Algorithm for Frequent Pattern Mining

## PRELIMINARIES

### TRANSACTIONS
Consider a set of transactions $T$ over items $I = \{ i_1, i_2, \ldots, i_m \}$.  A _transaction set_ or _itemset_ $X \in T$ satisfies $X \subset I$. That is $X$ is composed of a subset of items in $I$.

### ASSOCIATION RULE
Consider two itemsets $X$ and $Y$, disjoint subsets of $I$ &mdash; that is two transaction sets that do not share any transactions in common. Formally, $X \subset I$ and $Y \subset I$, where $X \cap Y  = \emptyset$.

An **association rule** is then of the form 

$$X \Rightarrow Y$$

that is the set of items in $X$ imply the set of items in $Y$.

### SUPPORT
_Support_ for an association rule is defined then as the number of transactions in $T$ such that $X \cup Y \cup D \neq \emptyset$. That is,

$$
\mathrm{supp}(X \implies Y) = { { \lvert \forall t \in T \mid t \cup X \cup Y \neq \emptyset \rvert } \over { \lvert T \rvert } }
$$

or equivalently 
$$
\mathrm{supp}(X \implies Y) = Pr(X, Y).
$$

We say _support_ is the percent of all transactions in $T$ that contain $X$ and $Y$.  We can also use support to determine the support for an itemset such that $\mathrm{supp}(X)$ is the proportion of transcations in $T$ that contain $X$. 

### CONFIDENCE
_Confidence_ for an association rule is defined as the number of transactions in $T$ such that 

$$
\mathrm{conf}(X \implies Y) = {
{
{ { \lvert \forall t \in T \mid t \cup X \cup Y \neq \emptyset \rvert } }
}
\over {  { \lvert \forall t \in T \mid t \cup X \neq \emptyset \rvert }   } 
}
$$

or equivalently 

$$
\mathrm{conf}(X \implies Y) = \mathrm{Pr}(Y|X).
$$

We say _confidence_ is the percent of transactions that contain $X$, which also contain $Y$.

### LIFT

Lift is defined as the ratio of the support of a rule to the support of each part of the rule.  That is,

$$ \mathrm{lift}( X \implies Y )  = { 
{ \mathrm{supp}(X \cup Y) }  \over { \mathrm{supp}(X) \times \mathrm{supp}(Y) } 
}
$$ 

If the lift is $\leq 1$ then this implies that $X$ and $Y$ are independent and should therefore not be considered good rules, as by definition rules should be dependent.

### CONVICTION
$$
\mathrm{conv}(X \implies Y) = { {1 - \mathrm{supp}(Y)} \over {1 - \mathrm{conf}(X \implies Y)} } $$

## Building an association rule with minimum support and and confidence 

Thus, the goal of our association rule is to find rules that meet a _minimum_ level of support and also a _minimum_ level of confidence.  That is, a rule is kept only if these minimum levels are met.  Formally, let $0 \leq sup_{min} \leq 1$ be the minimum support and $0 \leq conf_{min} \leq 1$ be the set for a rule, then a rule is kept iff

$$  sup(X \implies Y) \geq sup_{min} $$

and

$$ conf(X \implies Y) \geq conf_{min} $$

## APRIORI ALGORITHM SKETCH

**TWO STEPS** 
1. Find all itemsets that have minimum support (i.e. frequent itemsets).
2. Use these frequent itemsets to generate rules. 

**NOTE: A frequent itemset must reach the minimum support threshold.**

Let's explore the main intuiton behind why the algorithm works.  First, we will exploit the _a priori_ property, which states, 

> _any subsets of a frequent itemset are also themselves frequent itemsets_.


	C1  init-pass(T);  		
	F1  {f | f  C1, f.count/n  minsup};    // n: no. of transactions in T
	for (k = 2; Fk-1  ; k++) do		
		Ck  candidate-gen(Fk-1);
		for each transaction t  T do	
		    for each candidate c  Ck do  	
			if c is contained in t then			
			   c.count++; 
		    end	
		end	
	       Fk  {c  Ck | c.count/n  minsup}	
	end	
return F  k Fk;


# Implementation

Let's begin with a basic implementation of  `support` and `confidence`.

In [167]:
def support(rule_lhs, rule_rhs, transactions):
    transaction_count = len(transactions)
    rule_match_count = 0
    rule = set(rule_lhs).union(rule_rhs)
    
    for tid, v in transactions.iteritems():
        if rule.issubset(set(v)):
            rule_match_count += 1
    
    return (1.0)*rule_match_count / transaction_count

def confidence(rule_lhs, rule_rhs, transactions):
    rule = set(rule_lhs).union(rule_rhs)
    
    # let's get the transaction count that include rule_lhs
    rule_lhs_transactions = dict()
    for tid, v in transactions.iteritems():
        if set(rule_lhs).issubset(v):            
            rule_lhs_transactions[tid] = v
    
    if rule_lhs_transactions:
        # now let's get rule_lhs_transactions that have the rule
        rule_match_count = 0
        for tid, v in rule_lhs_transactions.iteritems():
            if rule.issubset(v):            
                rule_match_count += 1

        return (1.0)*rule_match_count / len(rule_lhs_transactions) 
    else:
        return 0.0

Now let's test the implementation on some sample data.

In [1]:
transactions = dict(
    t001=('beef', 'chicken', 'milk'), 
    t002=('beef', 'cheese'),
    t003=('cheese', 'boots'),
    t004=('beef', 'chicken', 'cheese'),
    t005=('beef', 'chicken', 'clothes', 'cheese', 'milk'),
    t006=('chicken', 'clothes', 'milk'),
    t007=('chicken', 'milk', 'clothes')
)

We should find that the support for `{chicken, clothes}` $\implies$ `{milk}` should be ${3 \over 7} = 0.42857$.

In [166]:
print support(['clothes', 'chicken'], ['milk'], transactions)
print confidence(['clothes', 'chicken'], ['milk'], transactions)

0.428571428571
1.0


In [42]:
# Algorithm Apriori(T)	
# 	C1  init-pass(T);  		
# 	F1  {f | f  C1, f.count/n  minsup};    // n: no. of transactions in T
# 	for (k = 2; Fk-1  ; k++) do		
# 		Ck  candidate-gen(Fk-1);
# 		for each transaction t  T do	
# 		    for each candidate c  Ck do  	
# 			if c is contained in t then			
# 			   c.count++; 
# 		    end	
# 		end	
# 	       Fk  {c  Ck | c.count/n  minsup}	
# 	end	
# return F  k Fk;
from collections import Counter

def apriori(T, min_sup=0.5):
    c_init = candidate_init(T)
    f_init = []
    
    for c, c_count in c_init.iteritems():
        if c_count / (1.* len(T)) >= min_sup:
            f_init.append([c])
    
    f_prior, f_k, k = f_init, [], 2
    while True:
        print f_prior
        candidate_k = candidate_gen(f_prior)
        for t in T:
            for c in candidate_k:
                print set(c), candidate_k
                if set(c).intersection(set(candidate_k)) == set(c):
                    print 'got it'
        break
    
    return f_init
    
def candidate_init(T):
    F_init = Counter()
    for t in T:
        F_init.update(t)
    return F_init

def candidate_gen(f):
    return f

In [43]:
T_ = [[1,2],[2,3,4],[1,2,3]]
apriori(T_)

[[1], [2], [3]]
set([1]) [[1], [2], [3]]


TypeError: unhashable type: 'list'

In [27]:
set([1,2,3]).intersection(set([9,1,2,5]))

{1, 2}

## Using Pandas to load a larger dataset

We can look in our `data/` directory and see a sample file `bank-data.csv` ... let's load that into Pandas and do some work.

In [3]:
import csv

with open('bank-data.csv') as fi:
    dr_fi = csv.reader(fi)
    
    transactions = {}
    for l in dr_fi:
        transactions[l[0]] = l[1:]

In [1]:
import pandas as pd

df = pd.read_csv("bank-data.csv")

In [2]:
df[(df.age>0) & (df.age<25)]['income'].mean()

13254.360229885053